In [15]:
import pandas as pd

# Ruta al archivo 
archivo = "/Users/severinodonate/Downloads/planet_osm_point_202505011028.csv"


# Leer el CSV con el separador correcto
df = pd.read_csv(archivo, sep=";", quotechar='"')

# Mostrar las primeras filas
df.head()


/var/folders/59/2dj5q51d63d6npy9bjv7ms0w0000gn/T/ipykernel_61193/3050376845.py:8: DtypeWarning: Columns (2,3,8,9,10,11,12,13,14,15,17,18,20,21,22,23,24,25,27,28,30,31,33,35,37,38,39,40,41,42,43,44,45,47,49,50,51,52,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, sep=";", quotechar='"')


,osm_id,access,addr:housename,addr:housenumber,addr:interpolation,admin_level,aerialway,aeroway,amenity,area,...,tower:type,tunnel,water,waterway,wetland,width,wood,z_order,tags,way
0,10588525931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""wikidata""=>""Q113945981"", ""ref:linz:place_id""=...",POINT (-18720227.738842003 -20788121.51645145)
1,3139785211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-19408756.422405742 -2251245.843130527)
2,3900761447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-19513236.833302896 -2408383.2044248176)
3,1875948156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-19943558.499686304 -1970091.1695145066)
4,5696879103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""website""=>""https://tighar.org/wiki/The_Seven_...",POINT (-19424905.385087833 -522188.9820661098)


In [16]:
import re

def parse_tags(tag_str):
    """
    Convierte un string de tags tipo 'key'=>"value" en un diccionario Python
    """
    if not isinstance(tag_str, str) or tag_str.strip() == "":
        return {}

    # Elimina comillas triples y extrae pares key=>value
    tag_pairs = re.findall(r'"([^"]+)"\s*=>\s*"([^"]*)"', tag_str)
    return {key: value for key, value in tag_pairs}


In [17]:
ejemplo = df['tags'].dropna().iloc[0]
print("Original:", ejemplo)
print("Parseado:", parse_tags(ejemplo))


Original: "wikidata"=>"Q113945981", "ref:linz:place_id"=>"48987", "archaeological_site"=>"sacrificial_site"
Parseado: {'wikidata': 'Q113945981', 'ref:linz:place_id': '48987', 'archaeological_site': 'sacrificial_site'}


In [18]:
df['tags_dict'] = df['tags'].apply(parse_tags)

In [19]:
# Aplicar la función a toda la columna
df['tags_dict'] = df['tags'].apply(parse_tags)

# Mostrar solo las columnas 'tags' y 'tags_dict' para comparar
df[['tags', 'tags_dict']].head()

,tags,tags_dict
0,"""wikidata""=>""Q113945981"", ""ref:linz:place_id""=...","{'wikidata': 'Q113945981', 'ref:linz:place_id'..."
1,NaN,{}
2,NaN,{}
3,NaN,{}
4,"""website""=>""https://tighar.org/wiki/The_Seven_...",{'website': 'https://tighar.org/wiki/The_Seven...


In [20]:
# Expandir los diccionarios en columnas
tags_expandidos = pd.json_normalize(df['tags_dict'])

# Unir las nuevas columnas al dataframe original
df_expandido = pd.concat([df.drop(columns=['tags_dict']), tags_expandidos], axis=1)

# Mostrar las primeras filas del nuevo dataframe
df_expandido.head()


,osm_id,access,addr:housename,addr:housenumber,addr:interpolation,admin_level,aerialway,aeroway,amenity,area,...,backcountry,name:nez,name:hur,ref:borden,name:lil,name:sal,name:nlaka'pamux,ref:corner,name:cr,destroyed:amenity
0,10588525931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3139785211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3900761447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1875948156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5696879103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print("Número de columnas:", df_expandido.shape[1])

Número de columnas: 1174


In [22]:
df_expandido.columns.tolist()

['osm_id',
 'access',
 'addr:housename',
 'addr:housenumber',
 'addr:interpolation',
 'admin_level',
 'aerialway',
 'aeroway',
 'amenity',
 'area',
 'barrier',
 'bicycle',
 'brand',
 'bridge',
 'boundary',
 'building',
 'capital',
 'construction',
 'covered',
 'culvert',
 'cutting',
 'denomination',
 'disused',
 'ele',
 'embankment',
 'foot',
 'generator:source',
 'harbour',
 'highway',
 'historic',
 'horse',
 'intermittent',
 'junction',
 'landuse',
 'layer',
 'leisure',
 'lock',
 'man_made',
 'military',
 'motorcar',
 'name',
 'natural',
 'office',
 'oneway',
 'operator',
 'place',
 'population',
 'power',
 'power_source',
 'public_transport',
 'railway',
 'ref',
 'religion',
 'route',
 'service',
 'shop',
 'sport',
 'surface',
 'toll',
 'tourism',
 'tower:type',
 'tunnel',
 'water',
 'waterway',
 'wetland',
 'width',
 'wood',
 'z_order',
 'tags',
 'way',
 'wikidata',
 'ref:linz:place_id',
 'archaeological_site',
 'website',
 'description',
 'name:ru',
 'fortification_type',
 'alt_na

In [25]:
# Guardar el dataframe en un nuevo archivo CSV
df_expandido.to_csv("/Users/severinodonate/Downloads/planet_osm_point_expandido.csv", index=False, sep=';', quoting=1)

In [26]:
# Número total de filas
total_filas = len(df_expandido)

# Crear resumen de cobertura por variable
resumen = (
    df_expandido.notna()
    .sum()
    .reset_index()
    .rename(columns={"index": "variable", 0: "valores"})
)

# Calcular porcentaje de cobertura
resumen["porcentaje"] = (resumen["valores"] / total_filas * 100).round(2)

# Ordenar por porcentaje descendente
resumen = resumen.sort_values(by="porcentaje", ascending=False).reset_index(drop=True)

resumen


,variable,valores,porcentaje
0,osm_id,148537,100.00
1,way,148537,100.00
2,historic,145608,98.03
3,tags,129970,87.50
4,archaeological_site,114287,76.94
...,...,...,...
1169,entrance_marker:subway,1,0.00
1170,species,6,0.00
1171,epoch,6,0.00
1172,razed:building,2,0.00


In [28]:
resumen.to_csv("/Users/severinodonate/Downloads/resumen_variables_points.csv", index=False)

In [4]:
import pandas as pd

# Cargar el archivo original
archivo = "/Users/severinodonate/Downloads/planet_osm_point_202505011028.csv"
df = pd.read_csv(archivo, sep=";", quotechar='"')

# Asegurarse de que la columna se llama 'tags'
columna_tags = [col for col in df.columns if 'tags' in col.lower()][0]

# Contar cuántas veces aparece 'wikidata' en esa columna
conteo = df[columna_tags].fillna('').str.contains('wikidata').sum()

print(f"La palabra 'wikidata' aparece en {conteo} filas de la columna '{columna_tags}'.")



/var/folders/59/2dj5q51d63d6npy9bjv7ms0w0000gn/T/ipykernel_62726/1870745011.py:5: DtypeWarning: Columns (2,3,8,9,10,11,12,13,14,15,17,18,20,21,22,23,24,25,27,28,30,31,33,35,37,38,39,40,41,42,43,44,45,47,49,50,51,52,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, sep=";", quotechar='"')


La palabra 'wikidata' aparece en 15412 filas de la columna 'tags'.
